<a href="https://colab.research.google.com/github/kolakows/CoGOR-21/blob/main/testing_2types_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/bhl

Mounted at /gdrive
/gdrive/MyDrive/bhl


In [2]:
import pandas as pd

train_unb = pd.read_csv('./unbalanced/train.csv')
test_unb = pd.read_csv('./unbalanced/test.csv')

train = pd.read_csv('./undersampled/train.csv')
test = pd.read_csv('./undersampled/test.csv')

In [3]:
import numpy as np

train_unb = train_unb.assign(ActivityType=np.where((train_unb['Activity'] == "STANDING") | (train_unb['Activity'] == "SITTING") | (train_unb['Activity'] == "LAYING"), 'IDLE', 'MOVING'))
train_unb = train_unb.drop(['subject', 'void()', 'Activity'], axis=1)
train = train.assign(ActivityType=np.where((train['Activity'] == "STANDING") | (train['Activity'] == "SITTING") | (train['Activity'] == "LAYING"), 'IDLE', 'MOVING'))
train = train.drop(['subject', 'void()', 'Activity'], axis=1)

test_unb = test_unb.assign(ActivityType=np.where((test_unb['Activity'] == "STANDING") | (test_unb['Activity'] == "SITTING") | (test_unb['Activity'] == "LAYING"), 'IDLE', 'MOVING'))
test_unb = test_unb.drop(['subject', 'void()', 'Activity'], axis=1)
test = test.assign(ActivityType=np.where((test['Activity'] == "STANDING") | (test['Activity'] == "SITTING") | (test['Activity'] == "LAYING"), 'IDLE', 'MOVING'))
test = test.drop(['subject', 'void()', 'Activity'], axis=1)

In [4]:
target = ['ActivityType']

train_unb_x, train_unb_y = train_unb.drop(target, axis = 1), train_unb[target]
test_unb_x, test_unb_y = test_unb.drop(target, axis = 1), test_unb[target]

train_x, train_y = train.drop(target, axis = 1), train[target]
test_x, test_y = test.drop(target, axis = 1), test[target]

In [5]:
import xgboost as xgb

model_unb = xgb.XGBClassifier()
model_unb.fit(train_unb_x, np.ravel(train_unb_y))

model = xgb.XGBClassifier()
model.fit(train_x, np.ravel(train_y))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [6]:
from sklearn.metrics import accuracy_score

y_pred_unb = model.predict(test_unb_x)
y_pred = model.predict(test_x)

accuracy_unb = accuracy_score(test_unb_y, y_pred_unb)
accuracy = accuracy_score(test_y, y_pred)
print("Accuracy for data with uneven rows: %.2f%%" % (accuracy_unb * 100.0))
print("Accuracy for data with even rows: %.2f%%" % (accuracy * 100.0))

Accuracy for data with uneven rows: 100.00%
Accuracy for data with even rows: 100.00%
